In [28]:
import requests
import re
import csv
import lxml.html as lh
import pandas as pd
import numpy as np 

In [29]:
from bs4 import BeautifulSoup

In [30]:
r=requests.get("https://www.mscorecard.com/mscorecard/courses.php?page=1&CourseName=&Country=USA&SubmitButton=Search", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})

In [31]:
c=r.content
soup=BeautifulSoup(c,"html.parser")

In [32]:
pre_url="https://www.mscorecard.com/mscorecard/courses.php?page="
post_url="&CourseName=&Country=USA&SubmitButton=Search"

In [33]:
page_nr=soup.find_all('select', {"class":"form-control"}) [1]
options = page_nr.find_all("option")
last_page=[y.text for y in options][-1]
first_page=[y.text for y in options][0]
header={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
#values = [o.get("value") for o in options]
#print(last_page)
#print (first_page)
#print(values)
#print(page_nr)
file = open('GC_course.csv', 'w')
writer = csv.writer(file)

In [34]:
# write title row
writer.writerow(['GC Name', 'Address', 'City', 'State', 'Zip', 'Phone', 'Email', 'Website'])

52

In [35]:
for page in range(int(first_page),int(last_page)+1):
    #print(pre_url+str(page)+post_url)
    gc=(pre_url+str(page)+post_url)
    #print(gc)
    r1=requests.get(gc)
    c1=r1.content
    soup=BeautifulSoup(c1,"html.parser")
    #print(soup.prettify())
    all_gc = [tag['href'] for tag in soup.find_all('a', {'class':"no-hover"})]
    #print(all_gc)
    l=[]
    for item in all_gc:
            df = pd.read_html("https://www.mscorecard.com/mscorecard/"+str(item))[0]
            r2=requests.get("https://www.mscorecard.com/mscorecard/"+str(item))
            c2=r2.content
            #Store the contents of the website under doc
            doc = lh.fromstring(c2)
            #Parse data that are stored between <tr>..</tr> of HTML
            tr_elements = doc.xpath('//tr')
            #Test for the number of holes
            if (len(tr_elements)) == 22:
                h=['2','3','4','5','6','7','8','9','10']
                ms=12
                mcr=13
                ws=14
                wcr=15
            if (len(tr_elements)) == 33:
                h=['2','3','4','5','6','7','8','9','10','12','13','14','15','16','17','18','19','20']
                ms=23
                mcr=24
                ws=25
                wcr=26
            #Create empty list
            tees=[]
            yards=[]
            holes=[]
            col=[]
            par=[]
            w_par=[]
            m_hcp=[]
            w_hcp=[]
            m_slope=[]
            m_rating=[]
            w_slope=[]
            w_rating=[]
            i=0
            j=2
            #For each row, store each first element (header) and an empty list
            for t in tr_elements[1]:
                i+=1
                name=t.text_content()
                #print ('%d:"%s"'%(i,name))
                #print(name)
                if name not in ['Hole','Par','Hcp']:
                    tees.append(name)
                col.append((name,[]))
            print('Tees', tees)
            #  front 9 and back 9 yardages
            def get_value(index):
                if (len(tr_elements)) == 22:
                    return list(np.concatenate([df.iloc[:,index][2:11].to_numpy()]))
                if (len(tr_elements)) == 33:
                    return list(np.concatenate([df.iloc[:,index][2:11].to_numpy(), df.iloc[:,index][12:21].to_numpy()]))
            yards = []
            for i in range(3,len(tees)+3):
                yards.append(get_value(i))
            print('Yards', yards) 
            # front 9 and back 9 MEN pars and handicaps
            for x in h:
                for t in (tr_elements[int(x)]):
                    name=t.text_content()
                    holes.append(name)
                par.append(holes[1])
                m_hcp.append(holes[2])
                holes.clear()
            print('Men par', par)
            print('Men Handicap', m_hcp)
            # front 9 and back 9 WOMEN pars and handicaps
            for x in h:
                for t in (tr_elements[int(x)]):
                    name=t.text_content()
                    holes.append(name)
                w_par.append(holes[len(tees)+3])
                w_hcp.append(holes[len(tees)+4])
                holes.clear()
            print('Women par', w_par)
            print('Women Handicap', w_hcp)
            soup=BeautifulSoup(c2,"html.parser")
            #print(soup.prettify())
            course = soup.find('div', class_='page-content')
            gc_name = course.h1.text
            addr = soup.find('div', class_='col-md-4')
            innerHtml = addr.decode_contents()
            addr_list =innerHtml.split('<br/>')
            x=re.findall(r'\d+',addr_list[1])
            str1=""
            for ele in x:
                str1 += ele
       
            #print (addr_list[1].replace(str1,'').split(',')[1])
            web = soup.find('div', class_='col-md-8')
            #print(gc_name)
            #print(addr.text.replace("USA",""))
            #print(web.a.text)
            #print(web.div.text.replace("phone",""))
            #print(web.select('[href^=mailto]'))
            #print(web.select('[href^=http]'))
            #print([a.text for a in web.select("a[href^=mailto:]")])
            
            ### start here
            gc_addr=addr_list[0]
            gc_city=addr_list[1].replace(str1,'').split(',')[0]
            try:
                gc_state=addr_list[1].replace(str1,'').split(',')[1]
            except:
                gc_state=''
            try:
                gc_zip=str1
            except:
                gc_zip=''
            try:
                gc_phone=web.div.text.replace("phone","")
            except:
                gc_phone=''
            try:
                gc_email=web.select('a[href^=mailto]')
            except:
                gc_email=''
            try:
                gc_web=web.select('a[href^=http]')
            except:
                gc_web=''
                
            print (str(gc_name) + ' ' + str(gc_addr) + ' ' + str(gc_city) + ' ' + str(gc_state) + ' ' + str(gc_zip) + ' ' + str(gc_phone) + ' ' + str(gc_email) + ' ' + str(gc_web))            

            #writer.writerow([gc_name.encode('utf-8'), gc_addr.encode('utf-8'), gc_city.encode('utf-8'), gc_state.encode('utf-8'), gc_zip.encode('utf-8'), gc_phone.encode('utf-8'), gc_email.encode('utf-8'), gc_web.encode('utf-8')])
            writer.writerow([gc_name, gc_addr, gc_city, gc_state, gc_zip, gc_phone, gc_email, gc_web])
   

Tees ['Black', 'Blue', 'White', 'Red (L)']
Yards [['451', '371', '200', '346', '442', '419', '241', '385', '541', '561', '387', '308', '456', '180', '584', '181', '405', '165'], ['401', '354', '164', '298', '408', '398', '210', '319', '511', '538', '372', '288', '448', '149', '560', '160', '379', '148'], ['346', '326', '152', '291', '363', '388', '200', '260', '415', '511', '361', '255', '386', '138', '548', '149', '330', '111'], ['251', '277', '117', '261', '325', '354', '166', '225', '355', '439', '278', '187', '325', '94', '489', '111', '269', '90']]
Men par ['4', '4', '3', '4', '4', '4', '3', '4', '5', '5', '4', '4', '4', '3', '5', '3', '4', '3']
Men Handicap ['2', '10', '18', '14', '6', '8', '16', '12', '4', '3', '9', '11', '5', '15', '1', '13', '7', '17']
Women par ['4', '4', '3', '4', '4', '4', '3', '4', '5', '5', '4', '4', '4', '3', '5', '3', '4', '3']
Women Handicap ['2', '10', '18', '14', '6', '8', '16', '12', '4', '3', '9', '11', '5', '15', '1', '13', '7', '17']
1757 Golf Cl

Tees ['Blue', 'White', 'Gold', 'Red']
Yards [['372', '413', '532', '385', '217', '526', '235', '346', '360', '394', '511', '354', '184', '340', '490', '400', '188', '365'], ['363', '403', '484', '368', '158', '513', '152', '337', '354', '386', '497', '341', '172', '303', '490', '374', '178', '330'], ['353', '347', '474', '353', '150', '451', '122', '283', '276', '344', '409', '322', '164', '245', '408', '334', '160', '315'], ['277', '339', '411', '287', '126', '438', '122', '277', '269', '332', '403', '306', '164', '245', '388', '294', '144', '283']]
Men par ['4', '4', '5', '4', '3', '5', '3', '4', '4', '4', '5', '4', '3', '4', '5', '4', '3', '4']
Men Handicap ['17', '2', '5', '13', '12', '9', '16', '11', '4', '1', '3', '7', '18', '14', '6', '8', '10', '15']
Women par ['4', '4', '5', '4', '3', '5', '3', '4', '4', '4', '5', '4', '3', '4', '5', '4', '3', '4']
Women Handicap ['17', '2', '5', '13', '12', '9', '16', '11', '4', '1', '3', '7', '18', '14', '6', '8', '10', '15']
A Ga Ming Golf 

Tees ['Black', 'Blue', 'White', 'Gold']
Yards [['357', '400', '533', '198', '347', '356', '528', '204', '366', '396', '518', '195', '532', '418', '403', '313', '183', '454'], ['340', '386', '522', '171', '330', '325', '501', '184', '340', '368', '504', '160', '509', '371', '358', '294', '167', '421'], ['319', '367', '486', '151', '309', '286', '471', '162', '328', '326', '482', '125', '500', '342', '333', '273', '144', '388'], ['300', '351', '449', '132', '251', '251', '434', '140', '298', '304', '404', '106', '422', '312', '308', '235', '121', '345']]
Men par ['4', '4', '5', '3', '4', '4', '5', '3', '4', '4', '5', '3', '5', '4', '4', '4', '3', '4']
Men Handicap ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
Women par ['4', '4', '5', '3', '4', '4', '5', '3', '4', '4', '5', '3', '5', '4', '4', '4', '3', '4']
Women Handicap ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
A. C. Read Golf Club NAS Pensac

KeyboardInterrupt: 

In [36]:
file.close()